<a href="https://colab.research.google.com/github/MilizCom/Agent_AI-App-Smart-Calender/blob/main/agentAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import requests
import json

# Ganti dengan API key kamu yang baru dan aman
api_key = "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"

# API endpoint untuk OpenRouter
url = "https://openrouter.ai/api/v1/chat/completions"


In [22]:
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime, timedelta, date
import re
import calendar

if not firebase_admin._apps:
    cred = credentials.Certificate("/content/drive/MyDrive/agent-ai-calender-firebase-adminsdk-fbsvc-cb966f3e13.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://agent-ai-calender-default-rtdb.asia-southeast1.firebasedatabase.app/'
    })


In [23]:

def call_openrouter_gpt(prompt: str, api_key: str, model: str = "openai/gpt-3.5-turbo-16k") -> str:
    """
    Memanggil model gpt-3.5-turbo-16k dari OpenRouter untuk menjawab prompt.

    Parameters:
    - prompt (str): Teks pertanyaan atau perintah yang ingin dikirim.
    - api_key (str): API key untuk autentikasi di OpenRouter.
    - model (str): Nama model (default: "openai/gpt-3.5-turbo-16k").

    Returns:
    - str: Jawaban dari model.
    """
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 200:
        result = response.json()
        answer = result["choices"][0]["message"]["content"]
        return answer
    else:
        raise Exception(f"Request gagal dengan status code {response.status_code}: {response.text}")


In [24]:
def ekstrak_waktu_kegiatan(prompt_user: str, nama_kegiatan: str, api_key: str):
    prompt_waktu_detail = (
        f"({prompt_user}) berdasarkan pesan ini dan judul '{nama_kegiatan}', "
        f"berikan perkiraan waktu mulai dan selesai kegiatan dalam format "
        f"'mulai:hh.mm selesai:hh.mm'. Jika tidak ada, berikan 'mulai:00.00 selesai:23.59'."
    )

    # Kirim ke LLM
    WaktuMulaiSelesai = call_openrouter_gpt(prompt_waktu_detail, api_key)

    print(f"Prompt yang dikirim ke LLM untuk deteksi waktu:\n{prompt_waktu_detail}")
    print(f"Hasil deteksi waktu mulai dan selesai:\n{WaktuMulaiSelesai}")

    # Gunakan regex untuk mendeteksi waktu
    match = re.search(
        r'(?:Mulai|mulai):\s*(\d{2}\.\d{2})\s+(?:selesai|Selesai):\s*(\d{2}\.\d{2})',
        WaktuMulaiSelesai,
        re.IGNORECASE
    )

    # Inisialisasi default
    mulai_hhmm = "00.00"
    selesai_hhmm = "23.59"

    if match:
        mulai_hhmm = match.group(1)
        selesai_hhmm = match.group(2)
        print(f"Waktu terdeteksi dan disimpan: Mulai={mulai_hhmm}, Selesai={selesai_hhmm}")
    else:
        print("Format waktu tidak terdeteksi seperti yang diharapkan. Menggunakan default.")

    return mulai_hhmm, selesai_hhmm

In [25]:
def extract_months(sentence: str) -> list:
    """
    Mengambil nama-nama bulan dalam kalimat yang diberikan.

    Parameters:
    - sentence (str): Kalimat yang akan diperiksa.

    Returns:
    - list: Daftar nama bulan yang ditemukan (bisa ada duplikasi).
    """
    # Daftar nama bulan dalam bahasa Indonesia
    nama_bulan = [
        "Januari", "Februari", "Maret", "April", "Mei", "Juni",
        "Juli", "Agustus", "September", "Oktober", "November", "Desember"
    ]

    # Cari nama bulan dalam kalimat (case insensitive)
    bulan_ditemukan = []
    for bulan in nama_bulan:
        if bulan.lower() in sentence.lower():
            bulan_ditemukan.append(bulan)

    return bulan_ditemukan

In [26]:
def extract_numbers(sentence: str) -> list:
    """
    Mengekstrak semua Frekuensi (integer) yang terdapat dalam kalimat.

    Parameters:
    - sentence (str): Kalimat yang akan diperiksa.

    Returns:
    - list: Daftar Frekuensi yang ditemukan (sebagai string).
    """
    # Gunakan regex untuk menemukan Frekuensi
    Frekuensi = re.findall(r'\d+', sentence)

    return Frekuensi

In [27]:
def get_holidays_by_month(target_month_name: str) -> dict:
    ref_holidays = db.reference('holidays')
    all_holidays_data = ref_holidays.get()

    if not all_holidays_data:
        print("No holiday data found in Firebase.")
        return {}

    filtered_holidays = {}

    # Daftar nama bulan dalam Bahasa Indonesia
    month_names_id = [
        "Januari", "Februari", "Maret", "April", "Mei", "Juni",
        "Juli", "Agustus", "September", "Oktober", "November", "Desember"
    ]

    try:
        target_month_num = month_names_id.index(target_month_name.capitalize()) + 1
    except ValueError:
        print(f"Month name '{target_month_name}' is invalid or not found. Returning empty data.")
        return {}

    print(f"Searching for holidays in month: {target_month_name} (Month number: {target_month_num})")

    # Iterasi seluruh data hari libur
    for date_key, holiday_title in all_holidays_data.items():
        try:
            # Parsing tanggal tanpa mengandalkan format 100% konsisten
            parsed_date = datetime.strptime(date_key, "%Y-%m-%d")
        except ValueError:
            # Coba parsing manual untuk format tidak normal, misal 2025-06-1 → ubah ke 2025-06-01
            parts = date_key.split("-")
            if len(parts) == 3:
                try:
                    year, month, day = int(parts[0]), int(parts[1]), int(parts[2])
                    parsed_date = datetime(year, month, day)
                except ValueError:
                    print(f"Warning: Invalid date value: {date_key}. Ignoring.")
                    continue
            else:
                print(f"Warning: Unknown holiday key format: {date_key}. Ignoring.")
                continue

        if parsed_date.month == target_month_num:
            formatted_date = parsed_date.strftime("%Y-%m-%d")
            filtered_holidays[formatted_date] = holiday_title

    return filtered_holidays

In [28]:
def extract_months(sentence: str) -> str | None: # Mengubah tipe return menjadi string atau None
    """
    Mengekstrak nama bulan pertama yang ditemukan dari kalimat dan mengembalikannya sebagai string.
    Jika tidak ada bulan ditemukan, mengembalikan None.
    """
    nama_bulan = [
        "Januari", "Februari", "Maret", "April", "Mei", "Juni",
        "Juli", "Agustus", "September", "Oktober", "November", "Desember"
    ]
    for bulan in nama_bulan:
        if bulan.lower() in sentence.lower():
            return bulan # Langsung kembalikan nama bulan sebagai string pertama yang ditemukan
    return None # Kembalikan None jika tidak ada bulan ditemukan


In [29]:
def tampilkan_libur_bulan(nama_bulan: str):
    """
    Menampilkan hari libur untuk bulan yang diberikan.

    Parameter:
    - nama_bulan (str): Nama bulan dalam bahasa Indonesia (contoh: "Juni", "Juli").

    Output:
    - Menampilkan daftar hari libur untuk bulan tersebut ke terminal.
    """

    bulan_target = extract_months(nama_bulan)
    libur_bulan_ini = get_holidays_by_month(bulan_target)

    print(f"\nHari libur di bulan {bulan_target}:")
    if libur_bulan_ini:
        for tanggal, judul in libur_bulan_ini.items():
            print(f"- {tanggal}: {judul}")
    else:
        print("Tidak ada hari libur ditemukan untuk bulan ini atau nama bulan tidak valid.")


In [30]:
from firebase_admin import db

def get_firebase_data_by_month(bulan: str) -> dict:
    # Referensi ke node agendas di Firebase
    ref = db.reference('agendas')
    all_agendas = ref.get()

    # Daftar bulan dalam bahasa Indonesia
    bulan_id = [
        "Januari", "Februari", "Maret", "April", "Mei", "Juni",
        "Juli", "Agustus", "September", "Oktober", "November", "Desember"
    ]

    try:
        bulan_num = bulan_id.index(bulan.capitalize()) + 1
        bulan_str = f"{bulan_num:02d}"  # misalnya "06" untuk Juni
    except ValueError:
        print(f"Bulan '{bulan}' tidak valid.")
        return {}

    # Filter hanya tanggal yang mengandung bulan tersebut
    filtered_agendas = {}

    if all_agendas:
        for tanggal, detail in all_agendas.items():
            if len(tanggal) == 10 and tanggal[5:7] == bulan_str:
                filtered_agendas[tanggal] = detail

    return {'agendas': filtered_agendas}


In [31]:
def generate_agenda_schedule(namaKegiatan, bulan, Frekuensi, DurasiJam, Waktu, api_key):
    example_json_output = '''{
  2025-06-20{
    "mulai": "08.00",
    "selesai": "11.00",
    "title": "Workshop UI/UX"
  }
}, '''

    # Gunakan triple-quote luar biasa dengan menyisipkan contoh JSON ke dalam prompt
    prompt = f"""Tahun ini adalah 2025.
Tolong bantu saya menjadwalkan kegiatan berikut ini:
- Nama Kegiatan: {namaKegiatan}
- Bulan: {bulan}
- Frekuensi: {Frekuensi} kali dalam sebulan
- Durasi: {DurasiJam} jam setiap kali
- Waktu yang diinginkan: {Waktu}

Perhatikan juga waktu kosong saya, jangan jadwalkan jika berbenturan dengan agenda lain, dan sesuaikan waktu kegiatan dengan tingkat konsentrasi ideal (misalnya pagi untuk konsentrasi tinggi).
Ini jadwal saya yang tidak boleh di tabrak:
{get_firebase_data_by_month(bulan)}

Jangan berikan penjelasan, cukup hasil akhir saja.

✅ Hasil akhirnya hanya dalam format JSON (tanpa penjelasan), dengan struktur seperti ini:
{example_json_output}
"""

    hasilGenerate = call_openrouter_gpt(prompt, api_key)
    return hasilGenerate


In [32]:
from firebase_admin import firestore

def ambil_user_berdasarkan_id(doc_id):
    db = firestore.client()
    doc_ref = db.collection('users').document(doc_id)
    doc = doc_ref.get()

    if doc.exists:
        data = doc.to_dict()
        user_data = {
            "email": data.get('email'),
            "nama": data.get('name'),
            "pekerjaan": data.get('pekerjaan'),
            "schedules": {}
        }

        schedules = data.get('schedules', {})
        for hari, waktu in schedules.items():
            user_data["schedules"][hari] = {
                "mulai": waktu.get('mulai'),
                "selesai": waktu.get('selesai')
            }

        return user_data
    else:
        return {"error": f"Dokumen dengan ID '{doc_id}' tidak ditemukan."}


In [33]:
user_id = 'V3iW7i7hClWD82UmIoXSDxOGVJw2'
detail_kerja_user = ambil_user_berdasarkan_id(user_id)
print(detail_kerja_user)

{'email': 'nhnakim@mail.com', 'nama': 'nabil hakim', 'pekerjaan': None, 'schedules': {}}


In [34]:
def buat_entri_kalender(detail_kerja_user: dict, prompt_user: str, api_key: str):
    """
    Mengirimkan prompt ke LLM untuk membuat entri kalender berdasarkan jadwal kerja dan aktivitas user.

    Parameters:
    - detail_kerja_user (dict): Jadwal kerja user (format dictionary).
    - prompt_user (str): Deskripsi aktivitas user (misal: "Saya akan mengerjakan tugas...").
    - api_key (str): API key untuk mengakses model LLM via OpenRouter.

    Returns:
    - hasil (str): Output dari LLM berupa entri kalender dalam format yang diminta.
    """

    prompt = f"""
Saya memiliki jadwal kerja sebagai berikut: {detail_kerja_user}

Berdasarkan jadwal kerja tersebut, buatkan entri kalender untuk aktivitas berikut:

"{prompt_user}"

Format hasil yang saya inginkan adalah sebagai berikut:
{{
    yyyy-mm-dd: {{
        "title": "judul kegiatan dari prompt_user",
        "mulai": "HH.MM",
        "selesai": "HH.MM"
    }}
}}

Pastikan jadwal kegiatan di jam kerja saya.
Jika waktunya bentrok dengan data ini: {get_firebase_data_by_month(bulan)}, sesuaikan dengan waktu luang saya di hari tersebut.
"""

    hasil = call_openrouter_gpt(prompt, api_key)
    print("Prompt berhasil dikirim ke LLM.\n")
    print("Hasil entri kalender:\n")
    print(hasil)
    return hasil


In [35]:
import json

def format_entri_kalender(hasil_generate) -> dict:
    """
    Memformat hasil JSON dari LLM agar memiliki struktur yang konsisten:
    {
        "yyyy-mm-dd": {
            "mulai": "HH.MM",
            "selesai": "HH.MM",
            "title": "Judul kegiatan"
        }
    }

    Parameters:
    - hasil_generate (str/dict): Output LLM dalam bentuk string JSON atau dict

    Returns:
    - result (dict): Dictionary yang sudah diformat dengan rapi
    """

    if not hasil_generate:
        print("⚠️ Tidak ada data hasil_generate yang diberikan.")
        return {}

    # Jika sudah dict, langsung pakai
    if isinstance(hasil_generate, dict):
        data = hasil_generate
    elif isinstance(hasil_generate, str):
        try:
            data = json.loads(hasil_generate)
        except json.JSONDecodeError as e:
            print("❌ Gagal mem-parsing hasil JSON:")
            print(e)
            return {}
    else:
        print(f"❌ Tipe data hasil_generate tidak didukung: {type(hasil_generate)}")
        return {}

    # Validasi data adalah dict
    if not isinstance(data, dict):
        print("❌ Data yang di-decode bukan dictionary.")
        return {}

    result = {}

    for date, info in data.items():
        # Pastikan info adalah dict
        if not isinstance(info, dict):
            print(f"⚠️ Abaikan entri tanggal {date} karena nilai bukan dict.")
            continue

        result[date] = {
            "mulai": str(info.get("mulai", "")),
            "selesai": str(info.get("selesai", "")),
            "title": str(info.get("title", ""))
        }

    print("✅ Hasil akhir entri kalender yang diformat:\n")
    print(json.dumps(result, indent=2))
    return result


In [36]:
def kirim_ke_firebase(data: dict, db):
    """
    Mengirim data entri kalender ke Firebase Realtime Database di bawah node 'agendas'.

    Parameters:
    - data (dict): Data yang ingin dikirim dalam format:
        {
            "yyyy-mm-dd": {
                "title": "...",
                "mulai": "...",
                "selesai": "..."
            },
            ...
        }
    - db: Objek Firebase (hasil dari firebase_admin.db)

    Returns:
    - None (data langsung dikirim ke Firebase)
    """

    try:
        agendas_ref = db.reference('agendas')

        for date, items in data.items():
            date_ref = agendas_ref.child(date)
            date_ref.set(items)

        print("✅ Data berhasil dikirim ke Firebase Realtime Database.")
    except Exception as e:
        print(f"❌ Gagal mengirim data ke Firebase: {e}")


In [37]:

def get_latest_prompt( node: str = 'prompt') -> str:
    """
    Mengambil prompt terbaru dari Firebase Realtime Database berdasarkan timestamp.

    Parameters:
    - service_account_path (str): Path ke file serviceAccountKey.json.
    - db_url (str): URL dari Firebase Realtime Database.
    - node (str): Nama node database tempat menyimpan prompt (default: 'prompt').

    Returns:
    - str: Prompt terbaru, atau None jika tidak ada data.
    """
    # Ambil data dari node
    ref = db.reference(node)
    data = ref.get()

    if data:
        # Urutkan berdasarkan key (diasumsikan timestamp)
        terbaru = sorted(data.items(), reverse=True)[0][1]
        return terbaru
    else:
        return None


In [38]:

# prompt_terbaru = get_latest_prompt()

# prompt_user = prompt_terbaru
# print(prompt_user)

# # 1. Ambil nama kegiatan
# prompt = f"({prompt_user}) berdasarkan pesan ini judul agenda nya apa? jawab judul nya saja"
# NamaKegiatan = call_openrouter_gpt(prompt, api_key)
# print(NamaKegiatan)


# # 3. Ekstrak waktu mulai dan selesai
# mulai, selesai = ekstrak_waktu_kegiatan(prompt_user, NamaKegiatan, api_key)

# # 4. Ambil bulan target
# prompt = f"({prompt_user}) berdasarkan pesan ini ({NamaKegiatan}) di bulan apa di laksanakan? (langsung jawab bulan nya saja)"
# bulan = call_openrouter_gpt(prompt, api_key).strip()
# bulan_target = extract_months(bulan)

# # 5. Ambil frekuensi kegiatan
# prompt = f"({prompt_user}) berdasarkan pesan ini ({NamaKegiatan}) ingin di laksanakan berapa kali selama bulan {bulan_target}? (langsung jawab angka saja)"
# Frekuensi = call_openrouter_gpt(prompt, api_key).strip()

# # 6. Ambil hari libur bulan tersebut
# hari_libur_di_bulan = tampilkan_libur_bulan(bulan_target)  # hanya tampilkan, jika ingin data: get_holidays_by_month(bulan_target)

# # 7. Deteksi jenis kegiatan: libur atau kerja
# prompt = f"({prompt_user}) berdasarkan pesan ini ini termasuk kegiatan liburan atau kerja? (jawab dengan 1 kata: libur atau kerja)"
# jenis_kegiatan = call_openrouter_gpt(prompt, api_key).strip()

# # 8. Deteksi durasi kegiatan
# prompt = f"({prompt_user}) berdasarkan pesan ini ({NamaKegiatan}) ingin di laksanakan dalam durasi berapa jam? (jawab angka saja, default 24)"
# DurasiJam = extract_numbers(call_openrouter_gpt(prompt, api_key))

# # 9. Deteksi waktu pelaksanaan: pagi/siang/sore/malam
# prompt = f"({prompt_user}) berdasarkan pesan ini ({NamaKegiatan}) bagusnya dilaksanakan kapan? (Jawab 1 kata: Pagi/Siang/Sore/Malam)"
# Waktu = call_openrouter_gpt(prompt, api_key).strip()

# # 10. Ambil data agenda bulan yang sama dari Firebase
# data_agenda_juni = get_firebase_data_by_month(bulan_target)
# print("📅 Agenda Bulan Tersebut:", data_agenda_juni)

# # 11. Buat agenda otomatis
# hasil_generate = generate_agenda_schedule(NamaKegiatan, bulan_target, Frekuensi, DurasiJam, Waktu, api_key)
# print("📌 Hasil Generate:\n", hasil_generate)

# # 12. Buat entri kalender berdasarkan hasil generate
# hasil_kalender = buat_entri_kalender(detail_kerja_user, prompt_user, api_key)

# # 13. Format hasil generate jadi dict Python
# data = formatted_data = format_entri_kalender(hasil_generate)
# print(data)
# # 14. Kirim ke Firebase
# kirim_ke_firebase(data, db)


In [39]:
from datetime import datetime
import time
def get_latest_prompt():
    ref = db.reference('prompt')
    data = ref.get()
    if not data:
        return None

    # Ubah key timestamp (string) menjadi datetime object
    parsed = []
    for key, value in data.items():
        try:
            waktu = datetime.strptime(key, "%Y-%m-%d %H:%M")
            parsed.append((waktu, value))
        except:
            continue  # skip jika gagal parsing

    # Ambil data dengan waktu paling baru
    if not parsed:
        return None
    parsed.sort(reverse=True)
    waktu_terbaru, prompt_terbaru = parsed[0]
    return prompt_terbaru, waktu_terbaru

last_timestamp = None

while True:
    try:
        hasil = get_latest_prompt()

        if hasil is None:
            print("⏳ Menunggu prompt...")
            time.sleep(5)
            continue

        prompt_user, current_timestamp = hasil

        if current_timestamp != last_timestamp:
            print(f"\n🆕 Prompt Baru Diterima: {prompt_user}")
            last_timestamp = current_timestamp

            # 1. Ambil nama kegiatan
            prompt = f"({prompt_user}) berdasarkan pesan ini judul agenda nya apa? jawab judul nya saja"
            NamaKegiatan = call_openrouter_gpt(prompt, api_key)
            print("📌 Nama Kegiatan:", NamaKegiatan)

            # 2. Ekstrak waktu mulai dan selesai
            mulai, selesai = ekstrak_waktu_kegiatan(prompt_user, NamaKegiatan, api_key)

            # 3. Ambil bulan target
            prompt = f"({prompt_user}) berdasarkan pesan ini ({NamaKegiatan}) di bulan apa di laksanakan? (langsung jawab bulan nya saja)"
            bulan = call_openrouter_gpt(prompt, api_key).strip()
            bulan_target = extract_months(bulan)

            # 4. Ambil frekuensi
            prompt = f"({prompt_user}) berdasarkan pesan ini ({NamaKegiatan}) ingin di laksanakan berapa kali selama bulan {bulan_target}? (langsung jawab angka saja)"
            Frekuensi = call_openrouter_gpt(prompt, api_key).strip()

            # 5. Ambil hari libur bulan itu
            hari_libur_di_bulan = tampilkan_libur_bulan(bulan_target)

            # 6. Deteksi jenis kegiatan
            prompt = f"({prompt_user}) berdasarkan pesan ini ini termasuk kegiatan liburan atau kerja? (jawab dengan 1 kata: libur atau kerja)"
            jenis_kegiatan = call_openrouter_gpt(prompt, api_key).strip()

            # 7. Durasi kegiatan
            prompt = f"({prompt_user}) berdasarkan pesan ini ({NamaKegiatan}) ingin di laksanakan dalam durasi berapa jam? (jawab angka saja, default 24)"
            DurasiJam = extract_numbers(call_openrouter_gpt(prompt, api_key))

            # 8. Waktu pelaksanaan
            prompt = f"({prompt_user}) berdasarkan pesan ini ({NamaKegiatan}) bagusnya dilaksanakan kapan? (Jawab 1 kata: Pagi/Siang/Sore/Malam)"
            Waktu = call_openrouter_gpt(prompt, api_key).strip()

            # 9. Ambil data agenda bulan yang sama
            data_agenda_juni = get_firebase_data_by_month(bulan_target)
            print("📅 Agenda Bulan Tersebut:", data_agenda_juni)

            # 10. Generate agenda
            hasil_generate = generate_agenda_schedule(NamaKegiatan, bulan_target, Frekuensi, DurasiJam, Waktu, api_key)
            print("📌 Hasil Generate:\n", hasil_generate)

            # 11. Entri kalender
            hasil_kalender = buat_entri_kalender(detail_kerja_user, prompt_user, api_key)

            # 12. Format dan kirim ke Firebase
            data = format_entri_kalender(hasil_generate)
            kirim_ke_firebase(data, db)

        else:
            print("🔁 Belum ada prompt baru...")

        time.sleep(5)

    except Exception as e:
        print("❗ Terjadi error:", e)
        time.sleep(10)


🆕 Prompt Baru Diterima: saya mau tahu jadwal saya di bulan ini
📌 Nama Kegiatan: Jadwal Bulan Ini
Prompt yang dikirim ke LLM untuk deteksi waktu:
(saya mau tahu jadwal saya di bulan ini) berdasarkan pesan ini dan judul 'Jadwal Bulan Ini', berikan perkiraan waktu mulai dan selesai kegiatan dalam format 'mulai:hh.mm selesai:hh.mm'. Jika tidak ada, berikan 'mulai:00.00 selesai:23.59'.
Hasil deteksi waktu mulai dan selesai:
Perkiraan jadwalnya adalah:

1. Meeting dengan tim kerja - mulai:09.00 selesai:10.00
2. Presentasi proyek kepada manajemen - mulai:13.00 selesai:15.00
3. Diskusi tentang strategi pemasaran - mulai:16.00 selesai:17.00
4. Makan malam bersama rekan kerja - mulai:19.00 selesai:20.00

Jadwal Bulan Ini
Waktu terdeteksi dan disimpan: Mulai=09.00, Selesai=10.00
Searching for holidays in month: September (Month number: 9)

Hari libur di bulan September:
- 2025-09-05: Maulid Nabi Muhammad SAW
- 2025-09-06: Hari Saraswati
📅 Agenda Bulan Tersebut: {'agendas': {'2025-09-01': {'mulai

KeyboardInterrupt: 